In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import retry
import os

In [2]:
#Try selenium

In [3]:
df_games = pd.DataFrame(columns=['Title',
                                 'Link',
                                 'Description',
                                 'Geek Rating', 
                                 'Avg Rating',
                                 'Num Voters', 
                                 'List Price', 
                                 'Amazon Price'])

### API Parser

In [4]:
games_list = pd.read_csv('boardgames_ranks.csv')

In [5]:
games_list.iloc[0:2000]

,id,name,yearpublished,rank,bayesaverage,average,usersrated,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank
0,224517,Brass: Birmingham,2018,1,8.41581,8.59842,44835,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1,161936,Pandemic Legacy: Season 1,2015,2,8.38155,8.52941,53143,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN
2,174430,Gloomhaven,2017,3,8.35758,8.59323,61748,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN
3,342942,Ark Nova,2021,4,8.33266,8.53712,42099,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
4,233078,Twilight Imperium: Fourth Edition,2017,5,8.24384,8.60632,23407,NaN,NaN,NaN,NaN,NaN,5.0,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,345976,System Gateway (fan expansion for Android: Net...,2021,1996,6.20677,9.17895,563,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN
1996,320718,Hidden Leaders,2022,1997,6.20591,6.82511,2893,NaN,NaN,NaN,651.0,NaN,NaN,NaN,NaN
1997,155695,Age of War,2014,1998,6.20588,6.45122,7917,NaN,NaN,NaN,718.0,NaN,NaN,NaN,NaN
1998,236217,Timebomb,2016,1999,6.20548,7.15437,1705,NaN,NaN,NaN,NaN,98.0,NaN,NaN,NaN


In [6]:
def requests_retry_session(
                            retries=5, 
                            backoff_factor=0.3,
                            status_forcelist=(500, 502, 504),
                            session=None,
                            ):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

In [7]:
# Папка для сохранения изображений
image_folder = 'game_images'
os.makedirs(image_folder, exist_ok=True)

# DataFrame для хранения данных
games_data = pd.DataFrame(columns=['id', 'name', 'description', 'image_name'])

# Функция для загрузки данных
def scrape_game_data(game_id):
    try:
        response = requests.get(f'https://www.boardgamegeek.com/boardgame/{game_id}', 
                                auth=('OlaviArg', 'I00magellan'), 
                                timeout=15)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Получение описания игры
            description_tag = soup.find('meta', property='og:description')
            description = description_tag.get('content') if description_tag else ''
            
            # Получение ссылки на изображение
            image_tag = soup.find('meta', property='og:image')
            image_url = image_tag.get('content') if image_tag else ''
            
            # Скачивание изображения и сохранение
            if image_url:
                image_name = f"{game_id}.jpg"
                image_path = os.path.join(image_folder, image_name)
                with open(image_path, 'wb') as f:
                    image_response = requests.get(image_url)
                    f.write(image_response.content)
            else:
                image_name = ''
            
            # Добавление данных в DataFrame
            games_data.loc[len(games_data)] = [game_id, game_name, description, image_name]
            
    except Exception as e:
        print(f"Error occurred while scraping game {game_id}: {e}")

# Пример использования
games_list_subset = games_list.iloc[0:2000]  # Ваш список игр

for i, row in games_list_subset.iterrows():
    game_id = row['id']
    game_name = row['name']
    scrape_game_data(game_id)
    time.sleep(5)  # Пауза между запросами

# Сохранение DataFrame с данными
games_data.to_csv('games_data.csv', index=False)

Error occurred while scraping game 182028: HTTPSConnectionPool(host='boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/182028 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027547AD89D0>, 'Connection to boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping game 266192: HTTPSConnectionPool(host='boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/266192 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027547A0B290>, 'Connection to boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping game 205637: HTTPSConnectionPool(host='www.boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/205637 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000275479F0690>, 'Connection to www.boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping game 341169: HTTP

Error occurred while scraping game 284653: HTTPSConnectionPool(host='www.boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/284653 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027547685550>, 'Connection to www.boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping game 396790: HTTPSConnectionPool(host='www.boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/396790 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027547A98AD0>, 'Connection to www.boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping game 332686: HTTPSConnectionPool(host='boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/332686 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027547F9E590>, 'Connection to boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping game 6658

Error occurred while scraping game 105134: HTTPSConnectionPool(host='www.boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/105134 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027547FFB290>, 'Connection to www.boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping game 15985: HTTPSConnectionPool(host='boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/15985 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002754809FD10>, 'Connection to boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping game 17405: HTTPSConnectionPool(host='boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/17405 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000275479FD850>, 'Connection to boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping game 252328: HTTPSCon

Error occurred while scraping game 283863: HTTPSConnectionPool(host='www.boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/283863 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027547A0BE90>, 'Connection to www.boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping game 30380: HTTPSConnectionPool(host='boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/30380 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027548017610>, 'Connection to boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping game 361545: HTTPSConnectionPool(host='boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/361545 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027547A8B410>, 'Connection to boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping game 154825: HTTPSC

Error occurred while scraping game 298065: HTTPSConnectionPool(host='www.boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/298065 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027547B3FED0>, 'Connection to www.boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping game 271529: HTTPSConnectionPool(host='boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/271529 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027547A349D0>, 'Connection to boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping game 244654: HTTPSConnectionPool(host='www.boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/244654 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027548175A50>, 'Connection to www.boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping game 2298

Error occurred while scraping game 151007: HTTPSConnectionPool(host='boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/151007 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000275480FE3D0>, 'Connection to boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping game 16395: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error occurred while scraping game 286063: HTTPSConnectionPool(host='www.boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/286063 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027548219950>, 'Connection to www.boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping game 291859: HTTPSConnectionPool(host='boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/291859 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0

Error occurred while scraping game 481: HTTPSConnectionPool(host='www.boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/481 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000275480AEF90>, 'Connection to www.boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping game 129051: HTTPSConnectionPool(host='www.boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/129051 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027547BDFB90>, 'Connection to www.boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping game 232361: HTTPSConnectionPool(host='www.boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/232361 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027547B7A550>, 'Connection to www.boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping game 19

Error occurred while scraping game 359402: HTTPSConnectionPool(host='www.boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/359402 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027548133710>, 'Connection to www.boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping game 298229: HTTPSConnectionPool(host='www.boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/298229 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027548130550>, 'Connection to www.boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping game 377470: HTTPSConnectionPool(host='www.boardgamegeek.com', port=443): Max retries exceeded with url: /boardgame/377470 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000275481A4410>, 'Connection to www.boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred while scraping g